In [1]:
import datetime
import pandas as pd
import re

def timestamp_para_datahora(timestamp):
    datahora = datetime.datetime.fromtimestamp(timestamp)
    return datahora

def formatar_dataframe(dataframe):
    # Converter a coluna "time" para valores numéricos e filtrar os valores nulos e não numéricos
    dataframe['time'] = pd.to_numeric(dataframe['time'], errors='coerce')
    dataframe = dataframe[~dataframe['time'].isna()]

    # Converter a coluna "time" para timestamp e criar a coluna "datetime"
    dataframe['datetime'] = dataframe['time'].astype(int).apply(timestamp_para_datahora)

    # Apagar a coluna "time", "summary", "icon", "cloudCover"
    dataframe = dataframe.drop(columns=['time', 'summary', 'icon', 'cloudCover'])

    # Ordena alfabeticamente as colunas
    dataframe = dataframe.sort_index(axis=1)

    # Loop pelas colunas do dataframe
    for coluna in dataframe.columns:

        # Pula a coluna "datetime"
        if coluna == 'datetime':
            continue

        # Define a coluna "datetime" como o índice do dataframe
        dataframe = dataframe.set_index('datetime')

        # Calcula a soma ou a diferença entre as colunas por intervalo de 1 minuto -> Errado
        df_agrupado = dataframe[coluna].resample('1T').sum()

        # Remove os valores NaN do resultado
        df_agrupado = df_agrupado.dropna()

        # Substitui a coluna original pelos valores agrupados
        dataframe[coluna] = df_agrupado

        # Reseta o índice do dataframe
        dataframe = dataframe.reset_index()

    # Remove as linhas desnecessaria, agora que o intervalo de cada linha é somente a cada 1 minuto
    dataframe = dataframe.iloc[::60, :]

    return dataframe


X_train = formatar_dataframe(pd.read_csv('../Files/HomeC.csv', delimiter=',', low_memory=False))

# Mostrar o resultado
X_train

C:\Users\Votogames\AppData\Local\Temp\ipykernel_3064\3008774110.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['datetime'] = dataframe['time'].astype(int).apply(timestamp_para_datahora)


,datetime,Barn [kW],Dishwasher [kW],Fridge [kW],Furnace 1 [kW],Furnace 2 [kW],Garage door [kW],Home office [kW],House overall [kW],Kitchen 12 [kW],...,gen [kW],humidity,precipIntensity,precipProbability,pressure,temperature,use [kW],visibility,windBearing,windSpeed
0,2016-01-01 02:00:00,2.332833,0.003817,2.271667,8.077850,15.513433,0.782767,14.508850,62.647783,0.036317,...,0.198417,37.18,0.0000,0.00,61013.28,2167.86,62.647783,600.00,16926.0,549.02
60,2016-01-01 02:01:00,2.350883,0.005950,4.531317,14.194200,13.208250,0.770167,2.597617,55.090017,0.033250,...,0.205300,36.60,0.0000,0.00,60973.92,2150.32,55.090017,600.00,17084.0,497.04
120,2016-01-01 02:02:00,2.066350,0.002550,3.569133,5.965467,16.364733,0.797917,2.604967,42.884167,0.034683,...,0.206900,36.78,0.0000,0.00,60959.34,2123.10,42.884167,600.00,16764.0,475.62
180,2016-01-01 02:03:00,2.051700,0.008300,3.624717,22.509200,14.473017,0.775517,3.900833,57.600767,0.036950,...,0.206850,38.40,0.0000,0.00,60962.52,2116.92,57.600767,600.00,15900.0,341.08
240,2016-01-01 02:04:00,1.909783,0.003617,2.106350,9.180000,11.450550,0.793200,2.603500,38.390183,0.039683,...,0.206350,38.50,0.0000,0.00,60948.90,2127.10,38.390183,600.00,15930.0,418.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503640,2016-01-06 21:54:00,1.934933,0.004500,2.654150,17.168133,29.179867,0.768850,2.511183,65.819917,0.032783,...,0.248250,42.72,0.0224,0.48,60766.52,2219.40,65.819917,562.80,11380.0,671.00
503700,2016-01-06 21:55:00,1.854133,0.003568,3.029500,17.836333,29.924017,0.751583,2.502683,76.595083,0.031617,...,0.246533,43.30,0.2588,10.92,60757.71,2255.05,76.595083,531.32,11858.0,718.17
503760,2016-01-06 21:56:00,1.919517,0.006000,2.948417,20.452367,28.952700,0.764567,2.515267,75.613833,0.033983,...,0.248483,48.06,0.6360,30.60,60727.78,2185.76,75.613833,514.58,11874.0,643.24
503820,2016-01-06 21:57:00,1.900783,0.005117,3.003783,16.342867,26.059167,0.777433,2.513983,66.021450,0.033250,...,0.249867,50.20,0.6235,30.60,60702.70,2134.85,66.021450,498.50,11510.0,441.70


Modelo

In [ ]:
from sklearn.ensemble import IsolationForest

# Treinando o modelo Isolation Forest
clf = IsolationForest(contamination=0.1, random_state=42)
clf.fit(X_train)

# Detectando anomalias nos dados de teste
y_pred = clf.predict(X_test) # -> Falta dados de teste